In [91]:
%%capture   
from tqdm import tqdm_notebook as tqdm  # capture must start the cell
tqdm().pandas()

# base iports
import numpy as np
import pandas as pd
import re
import time
import requests
import matplotlib.pyplot as plt
import seaborn as sns


#scraping imports
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

# NLP imports
import nltk
from nltk.corpus import gutenberg, stopwords
from nltk.collocations import *
from nltk import FreqDist
from nltk import word_tokenize
import string
import re
import gensim
from gensim.utils import simple_preprocess
from gensim.corpora import Dictionary
from gensim import corpora, models
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *

# language imports
from collections import Counter
# ! pip install langdetect
from langdetect import detect


# visualization imports
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import warnings
warnings.filterwarnings("ignore")

# Language

In [2]:
def ngo_filter(langs, data, strict, only):
    df_lang = data.copy()
    
    if only == False:
        if langs != 'any':
            if strict == True:
                df_lang.languages = df[df['languages'].notna() == True].languages.apply(lambda x: x if all([l in x for l in langs]) else np.nan)
            else:
                df_lang.languages = df[df['languages'].notna() == True].languages.apply(lambda x: x if any([l in x for l in langs]) == True else np.nan)    
            df_lang.dropna(subset=['languages'],inplace=True)
            
    if only == True:
        if langs != 'any':
            # languages
            if strict == True:
                df.languages = df[df['languages'].notna() == True].languages.apply(lambda x: x if all([l in x for l in langs]) else np.nan)
            else:
                df.languages = df[df['languages'].notna() == True].languages.apply(lambda x: x if any([l in x for l in langs]) == True else np.nan)    

    if langs == 'any':
        return dg_lang

In [4]:
nf = pd.read_csv('KEEP_df_with_keywords.csv')

In [7]:
nf.titles.tolist()

['Harry Potter and the Prisoner of Azkaban (Harry Potter, #3)',
 'The Perks of Being a Wallflower',
 'A Walk to Remember',
 'The Bad Beginning (A Series of Unfortunate Events, #1)',
 'Interpreter of Maladies',
 'The Reptile Room (A Series of Unfortunate Events, #2)',
 'Timeline',
 "Ender's Shadow (The Shadow Series, #1)",
 'Invisible Monsters',
 'Gardens of the Moon (Malazan Book of the Fallen, #1)',
 'Seabiscuit: An American Legend',
 'Angus, Thongs and Full-Frontal Snogging (Confessions of Georgia Nicolson, #1)',
 'The Elegant Universe: Superstrings, Hidden Dimensions, and the Quest for the Ultimate Theory',
 'Fruits Basket, Vol. 1',
 'The Girl Who Loved Tom Gordon',
 'Naruto, Vol. 01: Uzumaki Naruto (Naruto, #1)',
 'Cryptonomicon',
 'In the Heart of the Sea: The Tragedy of the Whaleship Essex',
 'House of Sand and Fog',
 'High Five (Stephanie Plum, #5)',
 'Mr. Maybe',
 'Chocolat (Chocolat, #1)',
 'Survivor',
 'Lucky',
 'The Testament',
 'Sputnik Sweetheart',
 'Tripwire (Jack Reacher

In [16]:
nf.columns

Index(['titles', 'authors', 'description', 'genre', 'id', 'num_ratings',
       'pages', 'ratings'],
      dtype='object')

In [34]:
# tfidf vectorize descriptions in dff
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(nf['description'])

# get dot product of tfidf matrix on the transposition of itself to get the cosine similarity
sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [37]:
def rec_w_filter(title, df, sim, length=None, popularity=None): 
    
    # find index of the title
    idx = df[df.titles == title].index[0]
    
    # create enumerated list of similarity scores and their indices
    sim_scores = list(enumerate(sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    sim_scores = sim_scores[1:100] # starts at 1 because the book selected will be most similar
   
    movie_indices = [i[0] for i in sim_scores]
    
    df = df_filter.iloc[movie_indices]
     
    return filter_df(length, popularity, df)                      

In [38]:
rec_w_filter('Bossypants', nf, sim)

UnboundLocalError: local variable 'df' referenced before assignment

In [30]:
def recs_test(title, df, list_length=11, suppress=True):
    '''
    Return recommendations based on a count vectorized BoW comprised of book author, genres and description.
    Takes in title, list length, a dataframe, a similarity matrix and an option to suppress output.
    '''
    
    recommended_books = []

    idx = df[df.titles == title].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:list_length+1].index)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        recommended_books.append(list(df.index)[i])
    
    if suppress == False:
        print (f"\n We recommend \n {recommended_books}")

    return df.loc[recommended_books].titles.tolist()    

In [32]:
recs_test('Timeline',nf)

[[1.         0.01453257 0.00646096 ... 0.00353464 0.00655978 0.00365781]
 [0.01453257 1.         0.01548294 ... 0.00288731 0.011797   0.01369037]
 [0.00646096 0.01548294 1.         ... 0.00335269 0.00985655 0.0145374 ]
 ...
 [0.00353464 0.00288731 0.00335269 ... 1.         0.00374979 0.        ]
 [0.00655978 0.011797   0.00985655 ... 0.00374979 1.         0.01560963]
 [0.00365781 0.01369037 0.0145374  ... 0.         0.01560963 1.        ]]
6

 We recommend 
 [3213, 1004, 3092, 116, 1125, 1271, 981, 812, 1569, 154, 1103]


['Pretty Girls',
 'The Walking Dead, Vol. 1: Days Gone Bye',
 'Smoke Gets in Your Eyes: And Other Lessons from the Crematory',
 'Thunderhead',
 'The Walking Dead, Vol. 2: Miles Behind Us',
 'Forever Odd (Odd Thomas, #2)',
 'Birthright',
 'Odd Thomas (Odd Thomas, #1)',
 'Grave Surprise (Harper Connelly, #2)',
 'In the Forests of the Night',
 'The Art of the Start: The Time-Tested, Battle-Hardened Guide for Anyone Starting Anything']

# Testing other stuff ~

In [92]:
df = pd.read_csv('last_50.csv')

In [93]:

df.head()

,titles,authors,ratings,num_ratings,id,img,description,genre,format,page,isbn,au_ge,key_words,bow
0,"Anne of Avonlea (Anne of Green Gables, #2)",L.M. Montgomery,4.23,"128,035 ratings",77390,https://i.gr-assets.com/images/S/compressed.ph...,"At sixteen, Anne is grown up...almost. Her gra...","Classics, Fiction, Young Adult, Historical, Hi...",paperback,304.0,9.780553e+12,lmmontgomery classics fiction youngadult histo...,funny bone heart peppery new schoolteacher som...,lmmontgomery classics fiction youngadult histo...
1,Ethan Frome,Edith Wharton,3.39,"101,461 ratings",5246,https://i.gr-assets.com/images/S/compressed.ph...,'He seemed a part of the mute melancholy lands...,"Classics, Fiction, Literature, Academic, School",paperback,189.0,9.780142e+12,edithwharton classics fiction literature acade...,tragic destinies part surface ethan frome work...,edithwharton classics fiction literature acade...
2,The Arabian Nights,Anonymous,4.05,"71,041 ratings",93101,https://i.gr-assets.com/images/S/compressed.ph...,The tales of told by Shahrazad over a thousand...,"Classics, Fiction, Fantasy, Short Stories",paperback,1049.0,9.780813e+12,anonymous classics fiction fantasy shortstories,full powerful sorcerers jinns imprisoned fabul...,anonymous classics fiction fantasy shortstorie...
3,Martin Eden,Jack London,4.43,"21,904 ratings",929782,https://i.gr-assets.com/images/S/compressed.ph...,Alternate cover edition of ISBN 9780140187724 ...,"Classics, Fiction, Literature, American",paperback,480.0,NaN,jacklondon classics fiction literature american,semiautobiographical martin eden story success...,jacklondon classics fiction literature america...
4,"A Girl of the Limberlost (Limberlost, #2)",Gene Stratton-Porter,4.15,"18,003 ratings",17567,https://i.gr-assets.com/images/S/compressed.ph...,"Set amid Indiana's vast Limberlost Swamp, this...","Classics, Fiction, Young Adult, Historical, Hi...",paperback,222.0,9.781576e+12,genestratton-porter classics fiction youngadul...,exactly environmentalist tract mother deforest...,genestratton-porter classics fiction youngadul...


In [94]:
from fuzzywuzzy import fuzz

In [180]:
str1 = 'The Arabian Nights'
str2 = 'night'

In [181]:
titles = df.titles.tolist()
idx = df.id.tolist()
check = [(t,i) for t,i in enumerate(df.id)]

In [182]:
check

[(0, 77390),
 (1, 5246),
 (2, 93101),
 (3, 929782),
 (4, 17567),
 (5, 259020),
 (6, 4711854),
 (7, 179639),
 (8, 38673),
 (9, 19337),
 (10, 469406),
 (11, 16802),
 (12, 259052),
 (13, 1841723),
 (14, 953232),
 (15, 1102987),
 (16, 141213),
 (17, 121178),
 (18, 847735),
 (19, 6226),
 (20, 608727),
 (21, 283860),
 (22, 32589),
 (23, 1420762),
 (24, 634043),
 (25, 238718),
 (26, 513574),
 (27, 3249668),
 (28, 1132604),
 (29, 1084577),
 (30, 183266),
 (31, 453751),
 (32, 1458550),
 (33, 971687),
 (34, 6815365),
 (35, 354665),
 (36, 176413),
 (37, 1624648),
 (38, 2689403),
 (39, 5520450),
 (40, 1014997),
 (41, 1573898),
 (42, 2025003),
 (43, 5981163),
 (44, 2985774),
 (45, 2916227),
 (46, 1774421),
 (47, 6435452),
 (48, 6362193),
 (49, 6385003)]

In [95]:
# check[:3]
# idz = [77390,5246,93101]
# # [idx in idz for idx in df.id]
# df[df.id in idz==True

keez = []
for k,v in df.iterrows():
    if v.id in idz:
        keez.append(k)

In [122]:

df.loc[df.titles == 'The Arabian Nights']

,titles,authors,ratings,num_ratings,id,img,description,genre,format,page,isbn,au_ge,key_words,bow
2,The Arabian Nights,Anonymous,4.05,"71,041 ratings",93101,https://i.gr-assets.com/images/S/compressed.ph...,The tales of told by Shahrazad over a thousand...,"Classics, Fiction, Fantasy, Short Stories",paperback,1049.0,9.780813e+12,anonymous classics fiction fantasy shortstories,full powerful sorcerers jinns imprisoned fabul...,anonymous classics fiction fantasy shortstorie...


In [145]:
title = df.loc[df.id == 93101].titles.tolist()[0]
title

'The Arabian Nights'

In [171]:
def return_query_pull(query,df):
    ''' Takes in a string and returns a df with 50 most similar titles '''
    matching_books = []

    for title in df.titles.tolist():
        book_id = df.loc[df.titles == title].id.tolist()[0]
        ratio_set = fuzz.token_set_ratio(title.lower(),query.lower())

        if ratio_set > 70:
            matching_books.append((book_id, ratio_set))

    matching_books.sort(key=lambda tup: tup[1], reverse=True)

    found_search = pd.DataFrame()
    return [book_id for (book_id, ratio) in matching_books]

    # for idx in [book_id for (book_id, ratio) in matching_books]:
    #     df.loc[df.id == idx]

    # return found_search






In [179]:
return_query_pull('nights',df)

[93101]